In [1]:
import os
import sys
from pyspark import SparkContext, SparkConf
from streaming.encounter_job import EncounterJob
from common.job import Job
spark=Job.getSpark()

In [2]:
job = EncounterJob()

In [3]:
job.run()

Encounter Streaming Job Started at = 2020-03-18 09:28:28.343346
Event received in window:  None
-------------------------------------------
Time: 2020-03-18 12:28:30
-------------------------------------------
Row(encounters=[112, 113, 114, 115, 116, 105, 106, 107, 108, 109, 110, 111], person_id=16)
Row(encounters=[528, 529, 530, 531, 532, 533, 534, 535, 536, 527], person_id=48)
Row(encounters=[720, 721, 722, 723, 724, 725, 726, 727, 728, 715, 731, 716, 732, 717, 718, 719], person_id=64)
Row(encounters=[912, 903, 904, 905, 906, 907, 908, 909, 910, 911], person_id=80)
Row(encounters=[1104, 1105, 1106, 1107, 1108, 1101, 1102, 1103], person_id=96)
Row(encounters=[1264, 1265, 1266, 1263], person_id=112)
Row(encounters=[1600, 1616, 1601, 1602, 1603, 1619, 1604, 1620, 1605, 1606, 1607, 1608, 1609, 1610, 1611, 1612, 1613, 1614, 1599, 1615], person_id=144)
Row(encounters=[1744, 1760, 1745, 1746, 1747, 1748, 1749, 1750, 1751, 1752, 1753, 1754, 1755, 1756, 1743, 1759], person_id=160)
Row(encount

-------------------------------------------
Time: 2020-03-18 12:30:26
-------------------------------------------

-------------------------------------------
Time: 2020-03-18 12:30:28
-------------------------------------------

-------------------------------------------
Time: 2020-03-18 12:30:30
-------------------------------------------

-------------------------------------------
Time: 2020-03-18 12:30:32
-------------------------------------------

-------------------------------------------
Time: 2020-03-18 12:30:34
-------------------------------------------

-------------------------------------------
Time: 2020-03-18 12:30:36
-------------------------------------------

-------------------------------------------
Time: 2020-03-18 12:30:38
-------------------------------------------

-------------------------------------------
Time: 2020-03-18 12:30:40
-------------------------------------------

-------------------------------------------
Time: 2020-03-18 12:30:42
----------

Check the status: http://localhost:4040/streaming/